### Moduļi

In [552]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from scipy.linalg import expm, logm

### Konstantes

In [553]:
# Aproksimācijas precizitāte
EPSILON = 1e-3

# Izmantojamo vārtu kopa
GATESET = {}

tempqc = QuantumCircuit(1)
tempqc.rz(np.pi / 4, 0)
GATESET['Rz'] = tempqc

tempqc = QuantumCircuit(1)
tempqc.h(0)
GATESET['H '] = tempqc
del tempqc

# Pārbauda, ka nebija problēmu ar vārtu definīcijām
print(Operator(GATESET['H ']).data)
print(Operator(GATESET['Rz']).data)

# Pauli matricas 
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], complex)
Y = np.array([[0, -1j], [1j, 0]], complex)
Z = np.array([[1, 0], [0, -1]], complex)
PAULI_BASE = [X, Y, Z]

[[ 0.70710678+0.j  0.70710678+0.j]
 [ 0.70710678+0.j -0.70710678+0.j]]
[[0.92387953-0.38268343j 0.        +0.j        ]
 [0.        +0.j         0.92387953+0.38268343j]]


### Saglabātās īsās ķēdes

In [554]:
# Īsas ķēdes ātrai piekļuvei aproksimācijai
SHORTHAND = []

# Sāk ar īsajām ķēdēm ar vienu vārtu
for (gate_name, gate_qc) in GATESET.items():
    gate_U = Operator(gate_qc).data
    SHORTHAND.append((gate_name, gate_qc, gate_U))

# Visas iespējamās kombinācijas no 2 līdz 5 vārtiem
for (i) in range(1, 5):
    for (name, qc, _) in SHORTHAND.copy():
        if (len(name.split()) == i):
            for (gate_name, gate_qc) in GATESET.items():
                new_name = name + ' ' + gate_name
                new_qc = qc.copy()
                new_qc.compose(gate_qc, [0], inplace=True)
                new_U = Operator(new_qc).data
                SHORTHAND.append((new_name, new_qc, new_U))

# debug : izdrukā visas īsās ķēdes
for (name, _, _) in SHORTHAND:
    print(name)

Rz
H 
Rz Rz
Rz H 
H  Rz
H  H 
Rz Rz Rz
Rz Rz H 
Rz H  Rz
Rz H  H 
H  Rz Rz
H  Rz H 
H  H  Rz
H  H  H 
Rz Rz Rz Rz
Rz Rz Rz H 
Rz Rz H  Rz
Rz Rz H  H 
Rz H  Rz Rz
Rz H  Rz H 
Rz H  H  Rz
Rz H  H  H 
H  Rz Rz Rz
H  Rz Rz H 
H  Rz H  Rz
H  Rz H  H 
H  H  Rz Rz
H  H  Rz H 
H  H  H  Rz
H  H  H  H 
Rz Rz Rz Rz Rz
Rz Rz Rz Rz H 
Rz Rz Rz H  Rz
Rz Rz Rz H  H 
Rz Rz H  Rz Rz
Rz Rz H  Rz H 
Rz Rz H  H  Rz
Rz Rz H  H  H 
Rz H  Rz Rz Rz
Rz H  Rz Rz H 
Rz H  Rz H  Rz
Rz H  Rz H  H 
Rz H  H  Rz Rz
Rz H  H  Rz H 
Rz H  H  H  Rz
Rz H  H  H  H 
H  Rz Rz Rz Rz
H  Rz Rz Rz H 
H  Rz Rz H  Rz
H  Rz Rz H  H 
H  Rz H  Rz Rz
H  Rz H  Rz H 
H  Rz H  H  Rz
H  Rz H  H  H 
H  H  Rz Rz Rz
H  H  Rz Rz H 
H  H  Rz H  Rz
H  H  Rz H  H 
H  H  H  Rz Rz
H  H  H  Rz H 
H  H  H  H  Rz
H  H  H  H  H 


### Ievads

In [555]:
TARGET = np.sqrt(0.5) * np.array([
    [1, 1],
    [1j, -1j]
])
# vai unitāra?
print("Vai unitāra?", np.allclose(TARGET @ TARGET.conj().T, I))
print("Aproksimējamais operators:")
print(TARGET)

Vai unitāra? True
Aproksimējamais operators:
[[0.70710678+0.j         0.70710678+0.j        ]
 [0.        +0.70710678j 0.        -0.70710678j]]


### Funkcijas

In [556]:
# Rekursīvais algoritms, kas balstās uz Soloveja-Kitājeva teorēmas
def solovay_kitaev_decomposition(U_target, depth=3):
    
    if (depth == 0):
        return shorthand_approximation(U_target)
    
    qc_approx = solovay_kitaev_decomposition(U_target, depth-1)
    U_approx = Operator(qc_approx).data

    A, B = gc_decomposition(U_target @ U_approx.conj().T)

    qc_A = solovay_kitaev_decomposition(A, depth-1)
    qc_B = solovay_kitaev_decomposition(B, depth-1)
    qc_A_inv = qc_A.inverse()
    qc_B_inv = qc_B.inverse()

    # Izveido jaunu ķēdi
    qc = QuantumCircuit(1)
    qc.compose(qc_A, [0], inplace=True)
    qc.compose(qc_B, [0], inplace=True)
    qc.compose(qc_A_inv, [0], inplace=True)
    qc.compose(qc_B_inv, [0], inplace=True)
    qc.compose(qc_approx, [0], inplace=True)

    return qc

# Funkcija, kas atrod labāko īso ķēdi dotajam operatoram
# izmanto rekursijas bāzes gadījumā
def shorthand_approximation(U_target):
    print("Ātrais tuvinājums operatoram:")
    print(U_target)

    min_error = float('inf')
    best_entry = SHORTHAND[0]
    for entry in SHORTHAND:
        error = np.linalg.norm(U_target - entry[2], 2)
        if error < min_error:
            min_error = error
            best_entry = entry
    print(f"Tuvākā ķēde {best_entry[0]} ar kļūdu {min_error}")

    return best_entry[1] # atgriež QuantumCircuit

# Grupas komutatora dekompozīcija
def gc_decomposition(U):
    print("---GC DEKOMPOZĪCIJA---")
    print("Grupas komutatora dekompozīcija operatoram:")
    print(U)

    # Noņem globālo fāzi
    phase = np.angle(np.linalg.det(U)) / 2
    U = U / np.exp(1j * phase)

    # Izvelk rotācijas asi un leņķi
    axis, phi = extract_axis_angle(U)

    print("angle", phi)
    
    # Sadala rotācijās ar ortonormālām asīm un vienādiem leņķiem
    theta = np.sqrt(phi)

    w = [0, 0, 1] # sāk ar Z asi
    if abs(np.dot(axis, w)) > 0.9:
        w = [0, 1, 0] # ja tuvu, izvēlas Y asi

    # Veido ortonormālo bāzi
    w = np.array(w)
    u = w - np.dot(w, axis) * axis
    u = u / np.linalg.norm(u)
    v = np.cross(axis, u)
    print(np.dot(axis, u))
    print("Ortonormālā bāze:")
    print("u:", u)
    print("v:", v)

    # Rotācijas matricas ap asīm u un v
    A = rotation_matrix(u, theta)
    B = rotation_matrix(v, theta)

    # Tests
    U_reconstructed = A @ B @ A.conj().T @ B.conj().T
    print("Atjaunotais operators no dekompozīcijas:")
    print(U_reconstructed)
    print("Atšķirība starp sākotnējo un atjaunoto operatoru:")
    print(np.linalg.norm(U - U_reconstructed, 2))
    print("---")
    
    return A, B


# Funkcija izvelk rotācijas asi un leņķi no unitāra operatora
def extract_axis_angle(R):

    # Aprēķina leņķi
    trace = np.trace(R)
    theta = np.arccos(np.real(trace) / 2) * 2

    # Aprēķina rotācijas asi
    A = (R - np.cos(theta / 2) * I) / (-1j * np.sin(theta / 2))
    nx = np.real(A[1, 0])
    ny = np.imag(A[1, 0])
    nz = np.real(A[0, 0])
    axis = np.array([nx, ny, nz]) / np.linalg.norm([nx, ny, nz])

    return axis, theta

def rotation_matrix(axis, theta):
    return expm(-1j * theta / 2 * (axis[0] * X + axis[1] * Y + axis[2] * Z))

### Main bloks

In [557]:
final_circuit = solovay_kitaev_decomposition(TARGET, 3)

final_operator = Operator(final_circuit).data
print("Final circuit operator:")
print(final_operator)

print("Final approximation error:", np.linalg.norm(TARGET - final_operator, 2))

final_circuit.draw()

Ātrais tuvinājums operatoram:
[[0.70710678+0.j         0.70710678+0.j        ]
 [0.        +0.70710678j 0.        -0.70710678j]]
Tuvākā ķēde H  Rz Rz ar kļūdu 0.7653668647301795
---GC DEKOMPOZĪCIJA---
Grupas komutatora dekompozīcija operatoram:
[[ 7.07106781e-01+7.07106781e-01j  2.29934717e-17-2.29934717e-17j]
 [-2.29934717e-17+2.29934717e-17j  7.07106781e-01+7.07106781e-01j]]
angle 4.2146848510894035e-08
0.0
Ortonormālā bāze:
u: [0. 0. 1.]
v: [ 0.  1. -0.]
Atjaunotais operators no dekompozīcijas:
[[ 1.00000000e+00-2.16315513e-12j  2.16315515e-12+2.10734240e-08j]
 [-2.16315509e-12+2.10734240e-08j  1.00000000e+00+2.16315513e-12j]]
Atšķirība starp sākotnējo un atjaunoto operatoru:
2.1073424245766337e-08
---
Ātrais tuvinājums operatoram:
[[0.99999999-0.00010265j 0.        +0.j        ]
 [0.        +0.j         0.99999999+0.00010265j]]
Tuvākā ķēde H  H  ar kļūdu 0.00010264848814508497
Ātrais tuvinājums operatoram:
[[ 9.99999995e-01+0.j -1.02648488e-04+0.j]
 [ 1.02648488e-04+0.j  9.99999995

┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├»
«   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H ├┤ H